## Day 14: Extended Polymerization 

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

```
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
```

The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.
Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

```
Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
```

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [86]:
from copy import deepcopy
from collections import Counter

def polymer_insertion(filename, steps):
    """
    Run polymer insertion process
    
    
    """
    
    # Read and parse polymer input
    with open(filename, "r") as infile:
        polymer_raw = [l.strip() for l in infile]
        
    polymer = polymer_raw[0]
    polymer_dict = {}
    for p in polymer_raw[2:]:
        polymer_instruction = p.split(" -> ")
        polymer_dict[polymer_instruction[0]] = polymer_instruction[1]
    
    poly_sequence = [polymer]
        
    for i in range(1, steps+1):
        
        # Use copy.deepcopy to be on the safe side
        polymer_new = deepcopy(polymer)
                
        # Offset to make sure insertion happens at right index
        # in the new polymer
        offset = 0
        
        # Look at length 2 substrings
        for j in range(0, len(polymer)-1):
            
            if polymer[j:j+2] in polymer_dict.keys():
                polymer_new = polymer_new[:(j+offset+1)] + polymer_dict[polymer[j:j+2]] + polymer_new[(j+offset+1):]
                offset += 1
                
        polymer = polymer_new
        
        poly_sequence.append(polymer)
    
    # take the quantity of the most common element and subtract the quantity of the least common element
    diff = max(Counter(polymer).values()) - min(Counter(polymer).values())
    print("Part 1: " + str(diff))
        
    return poly_sequence
    
polymer_insertion("day14-example.txt", 10);

Part 1: 1588


In [87]:
polymer_insertion("day14-input.txt", 10);

Part 1: 4244


## Part Two

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring 2192039569602 times) and the least common element is H (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [124]:
#### from copy import deepcopy
from collections import Counter

def polymer_insertion_part2(filename, steps, compare_part1 = False):
    """
    Run polymer insertion process - part 2
    
    
    """
    
    # Read and parse polymer input
    with open(filename, "r") as infile:
        polymer_raw = [l.strip() for l in infile]
        
    polymer = polymer_raw[0]
    polymer_dict = {}
    for p in polymer_raw[2:]:
        polymer_instruction = p.split(" -> ")
        polymer_dict[polymer_instruction[0]] = polymer_instruction[1]
    
    # Alternative approach - encode a polymer based on where
    # each pair appears
    
    # This assumes each pair in the polymer appears in the rule list
    
    # If this is not the case then need to add 'empty' rules for these new pairs
    # where no insertion takes place
    
    # Initialise polymer code
    
    polymer_code = dict.fromkeys(polymer_dict.keys(), 0)
    
    for j in range(0, len(polymer)-1):
        polymer_code[polymer[j:j+2]] += 1
    
    # Alternative dict where each pair is replaced with two pairs
    # by inserting into the middle
    polymer_dict_new = {}
    for x in polymer_dict.keys():
        polymer_dict_new[x] = [x[0] + polymer_dict[x], polymer_dict[x] + x[1]]

    letters = set(sum([[x[0], x[1]] for x in polymer_code.keys()], []))
    
    # For small step sizes get expected sequence from part 1 code
    if compare_part1:
        expected_sequence = polymer_insertion(filename, steps)
    
        #print(expected_sequence[0])

        expected_sequence_code = []
        for x in expected_sequence:    
            x_code = dict.fromkeys(polymer_dict.keys(), 0)
            for j in range(0, len(x)-1):
                x_code[x[j:j+2]] += 1
            expected_sequence_code.append(x_code)
        

    def count_letters(poly_code, letters_in):
        
        element_counts = dict.fromkeys(letters_in, 0)
        element_counts[polymer[0]] += 1

        element_counts[polymer[len(polymer)-1]] += 1

    
        for x in element_counts.keys():
            for y in poly_code.keys():
                if x in y:            
                    element_counts[x] += poly_code[y]
                    if y[0]==y[1]:
                        element_counts[x] += poly_code[y]
            
            element_counts[x] = element_counts[x]//2
                    
        
        return(element_counts)
    
    if compare_part1:
        expected_sequence_counts = []
        for ex in expected_sequence:
            element_counts = dict.fromkeys(letters, 0)
            for x in element_counts.keys():
                for y in ex:
                    if x in y:
                        element_counts[x] += 1

            expected_sequence_counts.append(element_counts)




    
        
    
    
    def print_polymer(poly_code, str_pre):
        print(str_pre)
        print([str(x) + " : " + str(polymer_code[x]) for x in polymer_code if polymer_code[x] > 0])
    
    #print_polymer(polymer_code, "Template: ")

    #print_polymer(expected_sequence_code[0], "Template (expected_sequence): ")

    #step_letters = count_letters(polymer_code, letters)
    #print(step_letters)
    #print(expected_sequence_counts[0])


    #print(polymer_code)

    # Now start the steps
    for i in range(1, steps+1):
        
        
        #print(polymer_code)
        

        #print(expected_sequence[i])
        
        polymer_code_new = deepcopy(polymer_code)
        
        #print(polymer_code_new)
        
        # For each pair, remove that pair and replace it with the instructed pairs
        for x in polymer_code.keys():
            
            if polymer_code[x] > 0:
                polymer_code_new[x] -= polymer_code[x]
                polymer_code_new[polymer_dict_new[x][0]] += polymer_code[x]

                polymer_code_new[polymer_dict_new[x][1]] += polymer_code[x]

        polymer_code = polymer_code_new  
        
        #print_polymer(polymer_code, F"After step {i}: ")

        #print_polymer(expected_sequence_code[i], F"After step {i} (expected_sequence): ")
        
        if compare_part1:
            step_letters = count_letters(polymer_code, letters)

            for x in step_letters.keys():
                if step_letters[x] != expected_sequence_counts[i][x]:
                    print(F"Unexpected count for {x} (got {step_letters[x]}, expected {expected_sequence_counts[i][x]})")

        
    step_letters = count_letters(polymer_code, letters)



    print(max(step_letters.values()) - min(step_letters.values()))

    
    return 0
    
polymer_insertion_part2("day14-example.txt", 10)



1588


0

In [125]:
polymer_insertion_part2("day14-input.txt", 40)

4807056953866


0